# [75.06 / 95.58] Organización de Datos
## Trabajo Práctico 1: Análisis Exploratorio de Datos

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**https://github.com/fdelmazo/7506-datos**

### Anexo: Creación de dataframes

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/events.csv", low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

### brands.csv

In [ ]:
checked = ['iphone', 'samsung', 'motorola', 'lenovo', 'sony', 'lg', 'ipad', 'asus', 'quantum', 'blackberry']
model_parsed = df['model'].dropna().map(lambda x: x.lower())
model_parsed = model_parsed.map(lambda x: x.split())

def find_brand(model):
    for str in model:
        if str in checked:
            return str
    return "other"


df['brand'] = model_parsed.map(find_brand)
df['brand'] = df['brand'].astype('category')

In [ ]:
df[['model', 'brand']].dropna().drop_duplicates().to_csv('data/brands.csv', index=False)

### os.csv

In [ ]:
df['operating_system_version'].unique()

In [ ]:
checked = ['windows', 'android', 'linux', 'mac', 'ios', 'ubuntu', 'chrome os', 'tizen', 'other']
os_version_parsed = df['operating_system_version'].dropna().map(lambda x: x.lower())

def find_os(os_version):
    for os in checked:
        if os in os_version:
            return os
    return "another"


df['operating_system'] = os_version_parsed.map(find_os)
df['operating_system'] = df['operating_system'].astype('category')

In [ ]:
df[['operating_system_version', 'operating_system']].dropna().head(10)
df[df['operating_system'] == 'another'][['operating_system_version', 'operating_system']].head(10)

In [ ]:
df['operating_system'].value_counts()

In [ ]:
df[['operating_system_version', 'operating_system']].dropna().drop_duplicates().to_csv('data/os.csv', index=False)

In [ ]:
df[['operating_system_version', 'operating_system']].head(10)

### browsers.csv

In [ ]:
checked = ['mobile safari', 'chrome mobile', 'ie mobile', 'firefox mobile', 'edge mobile', 'opera mobile',
           'mobile', 'chrome', 'android', 'opera', 'ie', 'firefox', 
           'facebook', 'samsung', 'chromium', 'edge', 'yandex', 'uc', 
           'other', 'safari', 'puffin', 'maxthon', 'vivaldi']
browser_version_parsed = df['browser_version'].dropna().map(lambda x: x.lower())

def find_browser(browser_version):
    for browser in checked:
        if browser in browser_version:
            return browser
    return "other"


df['browser'] = browser_version_parsed.map(find_browser)
df['browser'] = df['browser'].astype('category')

In [ ]:
df[(df['browser'] == 'mobile')][['browser_version', 'browser']]

In [ ]:
df[['browser_version', 'browser']].dropna().head(10)
display(df['browser'].value_counts())
df[(df['browser'] == 'another')][['browser_version', 'browser']]

In [ ]:
df[['browser_version', 'browser']].dropna().drop_duplicates().to_csv('data/browsers.csv', index=False)

In [ ]:
df[['browser_version', 'browser']].head(20)

#### Colors

# Funnels

Definimos una sesión como una serie de eventos por usuario, los cuales están todos con menos de 30 minutos de inactividad entre el actual y el anterior.

In [ ]:
funnel = df.sort_values(['person', 'timestamp'])

In [ ]:
FIRST_SESSION = 0.0000000000000001
funnel['time_diff'] = funnel.groupby('person')['timestamp'].diff()
funnel['time_diff'] = funnel['time_diff'].fillna(0)
funnel['time_diff_min'] = funnel['time_diff'] / np.timedelta64(1, 'm')

In [ ]:
THRESHOLD = 30 # minutes

funnel['new_session'] = funnel['time_diff_min'] > THRESHOLD
funnel['session_id'] = funnel.groupby('person')['new_session'].cumsum()
funnel['session_id'] = funnel['session_id'].astype('int')

In [ ]:
gb = funnel.groupby(['person', 'session_id'])

funnel['session_cumno'] = gb.cumcount()
funnel['session_total_events'] = gb['session_cumno'].transform(lambda x: x.size)
funnel['session_first'] = funnel['session_cumno'] == 0
funnel['session_last'] = funnel['session_cumno'] == (-1+funnel['session_total_events'])

In [ ]:
cols = ['person', 'timestamp', 'time_diff_min', \
        'session_id', 'event', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last']
funnel[cols]

In [ ]:
cols_csv = ['time_diff_min', \
        'session_id', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last']

funnel[cols_csv].to_csv('data/sessions.csv', index=False)